# Jigsaw - Agile Community Rules Classification
### https://www.kaggle.com/competitions/jigsaw-agile-community-rules

## LLM-Based Content Moderation with LoRA Fine-Tuning
This notebook builds upon our earlier zero-shot inference experiments using a Llama-1B model for Reddit content moderation. Previously, we explored:

Token-based inference using Unsloth + vLLM: https://www.kaggle.com/code/vinothkumarsekar89/jigsaw-acrc-llama-1b-infer-automodel-unsloth-vllm

Logit-based batch inference using vLLM: https://www.kaggle.com/code/vinothkumarsekar89/jigsaw-acrc-llama-1b-batch-infer-logits-vllm

These approaches relied solely on zero-shot prompting without any model fine-tuning.

In this notebook, we fine-tune the Llama-1B model using LoRA (Low-Rank Adaptation) on the provided training dataset to improve classification performance. We then compare the fine-tuned model’s performance against the previous zero-shot baselines using F1-score metrics on a validation set Further... 

Inference using fine-tuned model will be available here: WIP

## Install packages on Kaggle: Add-ons > Install Dependencies 

```bash
pip install pip3-autoremove
pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
pip install unsloth vllm
pip install scikit-learn
```

In [1]:
import kagglehub
import pandas as pd
import os
import glob

# Check if running on Kaggle
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    # Running on Kaggle
    base_path = "/kaggle/input/jigsaw-agile-community-rules/"
    df_train = pd.read_csv(f"{base_path}train.csv")
    df_test = pd.read_csv(f"{base_path}test.csv")
else:
    # Running locally
    base_path = "./data/final/"
    
    # Find all train files
    train_files = glob.glob(f"{base_path}df_train_*.csv")
    
    if train_files:
        train_dfs = [pd.read_csv(file) for file in train_files]
        df_train = pd.concat(train_dfs, ignore_index=True)
        print(f"Concatenated {len(train_files)} train files: {train_files}")
    else:
        raise FileNotFoundError(f"No train files found in {base_path}")
    
    # Find all test files
    test_files = glob.glob(f"{base_path}df_test_*.csv")
    if test_files:
        test_dfs = [pd.read_csv(file) for file in test_files]
        df_test = pd.concat(test_dfs, ignore_index=True)
        print(f"Concatenated {len(test_files)} test files: {test_files}")
    else:
        raise FileNotFoundError(f"No test files found in {base_path}")

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")
print(df_train.columns)
        
req_cols = ['subreddit', 'rule', 'positive_example_1', 'negative_example_1', 'positive_example_2',
           'negative_example_2', 'test_comment', 'violates_rule']
df_train = df_train[req_cols]
df_test = df_test[req_cols]

# Improved normalization function that handles quotes
def normalize_violates_rule(value):
    """Normalize violates_rule values to 'Yes' or 'No', handling quotes and case variations"""
    if pd.isna(value):
        return None
    
    # Convert to string, strip whitespace and quotes, then convert to lowercase
    normalized = str(value).strip().strip('"').strip("'").strip().lower()
    
    # Map all variations to Yes/No
    if normalized in ['yes', 'true', '1', 'y', 't']:
        return 'Yes'
    elif normalized in ['no', 'false', '0', 'n', 'f']:
        return 'No'
    else:
        return None  # Invalid values will become NaN

# Apply normalization to both datasets
for name, df in [("train", df_train), ("test", df_test)]:
    print(f"\n{name.capitalize()} - Before normalization:")
    print(f"Unique values:\n{df['violates_rule'].value_counts()}")
    
    df["violates_rule"] = df["violates_rule"].apply(normalize_violates_rule)
    
    print(f"\n{name.capitalize()} - After normalization:")
    print(f"Unique values:\n{df['violates_rule'].value_counts()}")
    
    before = len(df)
    df.dropna(subset=["violates_rule"], inplace=True)  # drop rows with invalid values
    after = len(df)
    print(f"Dropped {before - after} rows from {name} due to invalid 'violates_rule'")

# Check for missing values in other columns
print("\nChecking for missing values in other columns:")
for col in req_cols:
    dropped_rows = df_train[df_train[col].isna()].shape[0]
    print(f"{col}: {dropped_rows} rows would be dropped")
    
df_train = df_train[req_cols].dropna()
df_test = df_test[req_cols].dropna()

df_train = df_train.sample(frac=1, random_state=21).reset_index(drop=True)
df_test = df_test.sample(frac=1, random_state=21).reset_index(drop=True)

print(f"\nUsing path: {base_path}")
print("\nFinal results:")
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

# Verify final values
print(f"\nFinal train violates_rule values:\n{df_train['violates_rule'].value_counts()}")
print(f"\nFinal test violates_rule values:\n{df_test['violates_rule'].value_counts()}")

#df_train.to_csv("df_train_.csv", index=False)
# df_test.to_csv("df_test.csv", index=False)

print("\nSample of processed data:")
print(df_train.head(2))

Concatenated 6 train files: ['./data/final/df_train_01_gq_kdsr_14k.csv', './data/final/df_train_ds_cr_03.csv', './data/final/df_train_01_gq_cr_12k.csv', './data/final/df_train_02_ds_cr_35k.csv', './data/final/df_train_0.csv', './data/final/df_train_ds_fgkr_01.csv']
Concatenated 1 test files: ['./data/final/df_test_cr_12.csv']
Train shape: (105759, 11)
Test shape: (2000, 8)
Index(['subreddit', 'rule', 'positive_example_1', 'negative_example_1',
       'positive_example_2', 'negative_example_2', 'test_comment',
       'violates_rule', 'row_id', 'body', 'rule_violation'],
      dtype='object')

Train - Before normalization:
Unique values:
violates_rule
No       52529
Yes      51201
True      1031
False      998
Name: count, dtype: int64

Train - After normalization:
Unique values:
violates_rule
No     53527
Yes    52232
Name: count, dtype: int64
Dropped 0 rows from train due to invalid 'violates_rule'

Test - Before normalization:
Unique values:
violates_rule
Yes    1000
No     1000
Name:

## Set imports, variable names, parameters

In [2]:
## You may need this login if you want to upload model to kagglehub from local machine.
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    pass
else:
    kagglehub.login()

In [3]:
from unsloth import FastLanguageModel
import torch
import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

dtype = ( None )
load_in_4bit = False
load_in_8bit = False

### List of models
#unsloth/Llama-3.2-3B-Instruct
#unsloth/Qwen3-4B
#unsloth/Mistral-Nemo-Base-2407
# "unsloth/Qwen3-4B-unsloth-bnb-4bit"
# "unsloth/Qwen3-8B-unsloth-bnb-4bit"
# "unsloth/Qwen3-14B-unsloth-bnb-4bit"
# "unsloth/Qwen3-32B-unsloth-bnb-4bit"
 
######---Parameters to change---#######
#kaggle_model_path="/kaggle/input/model/transformers/1b-instruct/1"
local_model_path="unsloth/Qwen2.5-3B-Instruct"
base_model="Qwen2.5-3B-Instruct-unsloth"

max_seq_length = 3072
Rank=64
LORA_ALPHA=64
sample_len=int(df_train.shape[0])
max_iter_steps=-1
Epochs=1

## To upload to kagglehub (model name & variation version)
model_slug="Qwen2.5-3B-unsloth-jigsaw-acrc-lora"
variation_slug="01"
###--------------------------------###
train_parameters=f"_lora_fp16_r{Rank}_a{LORA_ALPHA}_s{sample_len}_e_{Epochs}_msl{max_seq_length}-"
#train_parameters += '-'.join(sorted([file.split('batch_')[1].split('_')[0] for file in train_files]))
train_parameters +="0-swap-cr123-kdsr1-gksr1"
print(train_parameters)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-20 23:14:56 [__init__.py:235] Automatically detected platform cuda.
Switching to PyTorch attention since your Xformers is broken.

Requires Flash-Attention version >=2.7.1,<=2.8.0 but got 2.8.3.
🦥 Unsloth Zoo will now patch everything to make training faster!
_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1


## Load model using unsloth

In [4]:

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    model_path=kaggle_model_path
else:
    model_path=local_model_path
        
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    #model_name="./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r128_a128_s40394_e_2_msl2048-0-cr12-qcr12345-kdsr4k-qkdsr1234-f0p610k--",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    load_in_8bit=load_in_8bit
)

print(model.dtype)


==((====))==  Unsloth 2025.10.1: Fast Qwen2 patching. Transformers: 4.55.4. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.695 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

torch.bfloat16


## Set LoRA Config

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = Rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = LORA_ALPHA,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 123,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

/home/vino/anaconda3/envs/kaggle/lib/python3.11/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
Unsloth 2025.10.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


## Prepare the dataset from train-data

In [6]:
import pandas as pd
from datasets import Dataset
import kagglehub
import os
import glob

def formatting_prompts_func(examples, tokenizer):
    """
    Format Reddit moderation dataset using tokenizer chat template
    """
    texts = []
    for i in range(len(examples['subreddit'])):
        # Create system message
        system_msg = f"""You are a really experienced moderator for the subreddit /r/{examples['subreddit'][i]}. 
Your job is to determine if the following reported comment violates the given rule. Answer with only Yes or No."""
        
        # Create user message with the rule and examples
        user_msg = f"""<rule>
{examples['rule'][i]}
</rule>

<examples>
<example>
<comment>{examples['positive_example_1'][i]}</comment>
<rule_violation>Yes</rule_violation>
</example>

<example>
<comment>{examples['positive_example_2'][i]}</comment>
<rule_violation>Yes</rule_violation>
</example>

<example>
<comment>{examples['negative_example_1'][i]}</comment>
<rule_violation>No</rule_violation>
</example>

<example>
<comment>{examples['negative_example_2'][i]}</comment>
<rule_violation>No</rule_violation>
</example>
</examples>

<test_comment>
{examples['test_comment'][i]}
</test_comment>"""
        
        # Assistant response is "Yes" or "No"
        assistant_msg = examples['violates_rule'][i]

        # Create messages list for chat template
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]


        formatted_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )

        texts.append(formatted_text)

    return {"text": texts}


def build_dataset(tokenizer):
    """
    Build both train and test datasets using tokenizer chat template
    """
   
    train_dataset = Dataset.from_pandas(df_train)
    train_dataset = train_dataset.map(
        lambda examples: formatting_prompts_func(examples, tokenizer), 
        batched=True
    )
    
    test_dataset = Dataset.from_pandas(df_test)
    test_dataset = test_dataset.map(
        lambda examples: formatting_prompts_func(examples, tokenizer), 
        batched=True
    )
    
    return train_dataset, test_dataset

In [7]:
dataset_train, dataset_test = build_dataset(tokenizer)

Map:   0%|          | 0/105759 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
# Check dataset sample output
dataset_train['text'][0]

'<|im_start|>system\nYou are a really experienced moderator for the subreddit /r/news. \nYour job is to determine if the following reported comment violates the given rule. Answer with only Yes or No.<|im_end|>\n<|im_start|>user\n<rule>\n"No Discriminatory Content:Do not engage in racism, sexism, homophobia, transphobia, ableism, anti-semitism, or any form of bigoted behavior or generalizations that demean or harm marginalized groups."\n</rule>\n\n<examples>\n<example>\n<comment>Of course it\'s happening in that neighborhood. You let those people move in and suddenly property values drop while crime rates skyrocket. They\'re just built different culturally, statistics don\'t lie about this stuff.</comment>\n<rule_violation>Yes</rule_violation>\n</example>\n\n<example>\n<comment>This is what happens when we have women in leadership positions. They\'re too emotional to handle real crises and should stick to what they\'re naturally good at - not running countries during wartime situations

In [9]:
import mlflow
mlflow.set_tracking_uri("ml_runs")  # Or your remote tracking URI
mlflow.set_experiment("lora-experiments")  # Your experiment name

<Experiment: artifact_location='/home/vino/ML_Projects/Jigsaw-ACRC-Kaggle/ml_runs/427319351079010507', creation_time=1759411832689, experiment_id='427319351079010507', last_update_time=1759411832689, lifecycle_stage='active', name='lora-experiments', tags={}>

## Setup SFT trainer & train the model

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_test,  # Add test dataset here
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = Epochs, 
        max_steps = max_iter_steps,
        learning_rate = 1e-4,
        fp16 = False,  # Disable FP16 to avoid gradient unscaling issues
        bf16 = is_bfloat16_supported(),  # Use BF16 instead if supported
        logging_steps = 10,
        optim = "adamw_torch",  # Use adamw_torch instead of adamw_8bit for better compatibility
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 123,
        eval_strategy = "steps", 
        eval_steps = 2000,
        max_grad_norm = 1.0,  # Add explicit gradient clipping
        # Save settings - save after each epoch
        save_strategy = "epoch",
        save_total_limit = 3,  # Keep only last 3 checkpoints to save space
        #load_best_model_at_end = True,
        #metric_for_best_model = "eval_loss",
        output_dir = "outputs",
        report_to = "mlflow", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/105759 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/2000 [00:00<?, ? examples/s]

[2025-10-20 23:15:40,671] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-10-20 23:15:40,915] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


/home/vino/anaconda3/envs/kaggle/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/vino/anaconda3/envs/kaggle/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [11]:
formatted_model = base_model.replace(".", "").replace("-", "_")
run_name=formatted_model+train_parameters
with mlflow.start_run(run_name=run_name):
    trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 105,759 | Num Epochs = 1 | Total steps = 6,610
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 119,734,272 of 3,205,672,960 (3.74% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
2000,0.763100,0.715204
4000,0.739900,0.657367
6000,0.721300,0.630210


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


## Save LoRA Adaptors

In [12]:
# #save merged 16bit
# import os
# dir_path = f"./lora/{run_name}"
# os.makedirs(dir_path, exist_ok=True)
# model.save_pretrained(f"{dir_path}/")  # Local saving
# tokenizer.save_pretrained(f"{dir_path}/")

## Save model in merged FP16 format (vllm compatible for inference)

In [13]:
#save merged 16bit
import os
dir_path = f"./lora/{run_name}_merged_fp16"
os.makedirs(dir_path, exist_ok=True)
model.save_pretrained_merged(dir_path, tokenizer, save_method = "merged_16bit")

Found HuggingFace hub cache directory: /home/vino/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...


Unsloth: Copying 2 files from cache to `./lora/Qwen25_3B_Instruct_unsloth_lora_f


Successfully copied all 2 files from cache to `./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16`


Unsloth: Preparing safetensor model files: 100%|█| 2/2 [00:00<00:00, 62601.55it/
Unsloth: Merging weights into 16bit: 100%|████████| 2/2 [00:58<00:00, 29.07s/it]


Unsloth: Merge process complete. Saved to `/home/vino/ML_Projects/Jigsaw-ACRC-Kaggle/lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16`


## Load model,LORA, Save model in merged FP16 format & convert to GPTQ int4

In [14]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel
# from auto_gptq import AutoGPTQForCausalLM,  BaseQuantizeConfig

# # 1. Load base model (consider using unquantized fp16 if available)
# base_model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
# model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# # 2. Load LoRA weights and merge
# lora_dir = "./lora/Qwen3_14B_unsloth_bnb_4bit_lora_fp16_r64_s26899_e_2_msl2048-0-1-2-4-5-7-9"
# model = PeftModel.from_pretrained(model, lora_dir)
# model = model.merge_and_unload()

# # 3. Save merged fp16 model
# save_dir = lora_dir + "_merged"
# model.save_pretrained(save_dir)
# tokenizer.save_pretrained(save_dir)

# # 4. Quantize merged model to GPTQ int4
# gptq_config =  BaseQuantizeConfig(bits=4, group_size=128)
# quantized_model = AutoGPTQForCausalLM.from_pretrained(
#     save_dir,
#     quantization_config=gptq_config
# )
# quantized_model.save_pretrained(lora_dir + "_GPTQ")


## Kagglehub login

In [15]:
# ## You may need this login if you want to upload model to kagglehub from local machine.
# if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
#     pass
# else:
#     kagglehub.login()

## Upload LoRA adaptors/model to kaggle

In [16]:
# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = dir_path
#LOCAL_MODEL_DIR = "./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s1704_e_2_msl2048-kr_1/"

MODEL_SLUG = model_slug # Replace with model slug.

# Learn more about naming model variations at
# https://www.kaggle.com/docs/models#name-model.
VARIATION_SLUG = variation_slug # Replace with variation slug.

kagglehub.model_upload(
  handle = f"vinothkumarsekar89/{MODEL_SLUG}/transformers/{VARIATION_SLUG}",
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'LoRA adapter')

Uploading Model https://www.kaggle.com/models/vinothkumarsekar89/Qwen2.5-3B-unsloth-jigsaw-acrc-lora/transformers/01 ...
Starting upload for file ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/merges.txt


Uploading: 100%|████████████████████████████| 1.67M/1.67M [00:02<00:00, 683kB/s]

Upload successful: ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/merges.txt (2MB)
Starting upload for file ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/tokenizer_config.json


Uploading: 100%|███████████████████████████| 7.36k/7.36k [00:00<00:00, 9.66kB/s]

Upload successful: ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/tokenizer_config.json (7KB)
Starting upload for file ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/model-00001-of-00002.safetensors


...

Uploading: 100%|███████████████████████████| 2.51k/2.51k [00:00<00:00, 3.24kB/s]

Upload successful: ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/chat_template.jinja (2KB)
Starting upload for file ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/config.json


Uploading: 100%|███████████████████████████| 1.56k/1.56k [00:00<00:00, 1.71kB/s]

Upload successful: ./lora/Qwen25_3B_Instruct_unsloth_lora_fp16_r64_a64_s105759_e_1_msl3072-0-swap-cr123-kdsr1-gksr1_merged_fp16/config.json (2KB)


Your model instance version has been created.
Files are being processed...
See at: https://www.kaggle.com/models/vinothkumarsekar89/Qwen2.5-3B-unsloth-jigsaw-acrc-lora/transformers/01


In [17]:
import torch

def predict_violation(model, tokenizer, example):

    # Construct system message
    system_msg = (
        f"You are an expert moderator for /r/{example['subreddit']}. "
        "Your task is to determine if a comment violates the rule. "
        "Answer strictly with 'Yes' or 'No'."
    )

    # Construct user message
    user_msg = (
        f"Rule: {example['rule']}\n\n"

        f"Examples of non-violations:\n"
        f"1. {example['negative_example_1']} -> No\n"
        f"2. {example['negative_example_2']} -> No\n\n"

        f"Examples of violations:\n"
        f"1. {example['positive_example_1']} -> Yes\n"
        f"2. {example['positive_example_2']} -> Yes\n\n"

        f"Now, decide if the following test comment violates the rule:\n"
        f" Test Comment: {example['test_comment']}"
    )

    # Prepare message structure
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]

    # Format prompt using chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True  # add prompt for model to complete
    )

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the generated answer (after the prompt)
    generated_text = response[len(prompt):].strip()

    return response


In [18]:
example_input = {
    "subreddit": "AskReddit",
    "rule": "No personal attacks or hate speech.",
    "negative_example_1": "I disagree with your opinion, but that's okay.",
    "negative_example_2": "That's an interesting take, thanks for sharing.",
    "positive_example_1": "You're an idiot, your opinion is worthless.",
    "positive_example_2": "People like you are what's wrong with the world.",
    "test_comment": "You're so dumb it hurts to read your posts."
}

prediction = predict_violation(model, tokenizer, example_input)
print("Prediction:", prediction)

Prediction: system
You are an expert moderator for /r/AskReddit. Your task is to determine if a comment violates the rule. Answer strictly with 'Yes' or 'No'.
user
Rule: No personal attacks or hate speech.

Examples of non-violations:
1. I disagree with your opinion, but that's okay. -> No
2. That's an interesting take, thanks for sharing. -> No

Examples of violations:
1. You're an idiot, your opinion is worthless. -> Yes
2. People like you are what's wrong with the world. -> Yes

Now, decide if the following test comment violates the rule:
 Test Comment: You're so dumb it hurts to read your posts.
assistant
Yes
